In [1]:
import os
import glob

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from sklearn.cluster import KMeans

import sys
sys.path.append('..')

import modules.plotting as pl
import modules.general as gen
import modules.gait_metrics as gm
import modules.assign_sides as asi
import modules.linear_algebra as lin

In [2]:
%matplotlib auto

Using matplotlib backend: MacOSX


In [4]:
file_name = '2014-12-08_P004_Post_004'

In [5]:
best_pos_dir = os.path.join('..', 'data', 'kinect', 'best pos')
hypo_dir = os.path.join('..', 'data', 'kinect', 'processed', 'hypothesis')

hypo_paths = glob.glob(os.path.join(hypo_dir, '*.pkl'))
best_pos_paths = glob.glob(os.path.join(best_pos_dir, '*.pkl'))

hypo_paths = [x for x in hypo_paths if file_name in x]
best_pos_paths = [x for x in best_pos_paths if file_name in x]

df_hypo = pd.read_pickle(hypo_paths[0])
df_head_feet = pd.read_pickle(best_pos_paths[0])

## Split trial into separate passes

In [29]:
# Convert all position vectors to float type
# so they can be easily input to linear algebra functions
df_head_feet = df_head_feet.applymap(pd.to_numeric)

# Cluster frames with k means to locate the 4 walking passes
frames = df_head_feet.index.values.reshape(-1, 1)
k_means = KMeans(n_clusters=4, random_state=0).fit(frames)

# Sort labels so that the frames are in temporal order
labels = gen.map_sort(k_means.labels_)

# DataFrames for each walking pass in a trial
pass_dfs = gen.group_by_label(df_head_feet, labels)

In [30]:
df_pass = list(pass_dfs)[0]

In [31]:
frames = df_pass.index.values

In [32]:
_, direction_pass = gm.direction_of_pass(df_pass)

# Assign correct sides to feet
df_assigned = asi.assign_sides_pass(df_pass, direction_pass)

## Foot signal

In [33]:
signal_l = gm.foot_signal(df_assigned.L_FOOT, df_assigned.R_FOOT, direction_pass)

In [34]:
foot_dist = (df_assigned.L_FOOT - df_assigned.R_FOOT).apply(np.linalg.norm)

In [35]:
x = lin.line_coordinate_system(np.array([0, 0, 0]), direction_pass, np.stack(df_assigned.R_FOOT))

series_r = pd.Series(x, index=frames)

In [36]:
plt.plot(foot_dist)

In [37]:
plt.plot(series_r)